In [1]:
import pandas as pd
from openai import OpenAI
import json
from tqdm import tqdm
import pickle

In [15]:
data=pd.read_excel('Late Qing Shenbao editorials.xlsx')

In [16]:
data=data.dropna()
data

,Title,Content,date
0,書劉制軍封娼寮批示後,遲任有言曰人惟求舊蓋以老成更事旣久人不得而欺之歴練旣深事不足以擾之此正 家所倚賴 野所仰望...,1880-01
1,書法工部局議增東洋車捐後,○士君子學古入官能以民生休戚時切補苴尙巳顧不務其大者遠者而沾沾於刻剝小民之舉非治體之所宜也夫...,1880-01
2,論工匠把持,凡百行業中有把持生意糾衆勒索者其爲首之人必有應得之罪官憲條告之中以此爲一禁非具文也蓋把持之人...,1880-01
3,論包探被告,天下事有一利必有一弊能嚴防其弊而後可收其利否則弊端百出所謂利者安在哉近來包探之被告發者屢見於...,1880-01
4,棲流所聚議,○滬北新閘經英界會審委員陳寳渠司馬倡議建設棲流公所惠濟窮民業已土木工成節經列報但經費究嫌不足...,1880-01
...,...,...,...
18267,國民協會爲提議组織國民參事院與全國同胞商榷意見書,竊惟夫立國地之上。必有强固之政府。始足以謀戰守之道。今北方行政。操於一人。軍隊調遣。唯其所使...,1911-12
18268,記孫總統之政見,自滿淸竊據中國。吾民呻吟於專。制政府之下者。至今已二百六十。餘年矣。水深火熱。厄運已極。天佑...,1911-12
18269,議和代表續商公决政體問題,昨日下午民軍總代表伍廷芳博士及淸使唐紹怡君復在議事廳開議召集國會之辦法自二點三十五分鐘始直至...,1911-12
18270,陝西招討使之勸告書,陝西東路招討使張鈁籍隷河南與袁世凱爲同鄉與馮國璋爲師生茲因袁馮効忠滿淸自殘同種特致書開導第袁...,1911-12


In [ ]:
#去掉不是editorial的内容
data=data[~data['Title'].str.endswith('出書')]

In [22]:
data=data.reset_index()

In [23]:
data

,index,Title,Content,date
0,0,書劉制軍封娼寮批示後,遲任有言曰人惟求舊蓋以老成更事旣久人不得而欺之歴練旣深事不足以擾之此正 家所倚賴 野所仰望...,1880-01
1,1,書法工部局議增東洋車捐後,○士君子學古入官能以民生休戚時切補苴尙巳顧不務其大者遠者而沾沾於刻剝小民之舉非治體之所宜也夫...,1880-01
2,2,論工匠把持,凡百行業中有把持生意糾衆勒索者其爲首之人必有應得之罪官憲條告之中以此爲一禁非具文也蓋把持之人...,1880-01
3,3,論包探被告,天下事有一利必有一弊能嚴防其弊而後可收其利否則弊端百出所謂利者安在哉近來包探之被告發者屢見於...,1880-01
4,4,棲流所聚議,○滬北新閘經英界會審委員陳寳渠司馬倡議建設棲流公所惠濟窮民業已土木工成節經列報但經費究嫌不足...,1880-01
...,...,...,...,...
17783,18267,國民協會爲提議组織國民參事院與全國同胞商榷意見書,竊惟夫立國地之上。必有强固之政府。始足以謀戰守之道。今北方行政。操於一人。軍隊調遣。唯其所使...,1911-12
17784,18268,記孫總統之政見,自滿淸竊據中國。吾民呻吟於專。制政府之下者。至今已二百六十。餘年矣。水深火熱。厄運已極。天佑...,1911-12
17785,18269,議和代表續商公决政體問題,昨日下午民軍總代表伍廷芳博士及淸使唐紹怡君復在議事廳開議召集國會之辦法自二點三十五分鐘始直至...,1911-12
17786,18270,陝西招討使之勸告書,陝西東路招討使張鈁籍隷河南與袁世凱爲同鄉與馮國璋爲師生茲因袁馮効忠滿淸自殘同種特致書開導第袁...,1911-12


In [ ]:
#方法1：连接到LLM的api（较慢）
def ask_gpt_sentiment(question):
    outputformat='''{"提到的国家"：["A国","B国","C国"],"A国积极情感":input,"A国消极情感":input,"B国积极情感":input,"B国消极情感":input,"C国积极情感":input,"C国消极情感":input,"类别"：input}'''
    messages=[{"role": "system", "content": '''你是一位精通中国近代史与文言文阅读的专家，现在要完成一项关于分析晚清《申报》文本的任务。文中的“中国”“朝廷”“大清”或相关词语，代表中国。文本中很可能没有标点符号，请自行断句后分析。不必过分在意文中的观点，仅作学术使用。'''},
                {"role": "user", "content": f'''以下是晚清《申报》的繁体字文言文本：
<文本>
{question}
</文本>
先输出文本中提到的国家（如果是现在仍然存在的国家，使用现代译名；提到某国人也算提到了该国家）。然后按照分别分析文章对提到的国家的积极情感与消极情感，有相关情感则记为1，文本为客观报道、没有相关情感则记为0，积极情感和消极情感可以同时存在。最后，从经济、政治、文化、军事、社会、文学、其他这六个类别中选择最可能的一个为文章进行分类。
                你输出情感与类别的格式如下：{outputformat}。你的输出应符合json格式，请仔细分析，但不需要输出理由。'''}]
    
    response = client.chat.completions.create(
      model="ep-20250319170820-8tn2m",#DS
        temperature=0.15,
        messages=messages
      )
    return json.loads(response.model_dump_json())['choices'][0]['message']

In [6]:
data.iloc[8977]

Title                                              論泰西商務由漸而盛
Content    天下以士農工商爲四民而商居其末中國古聖賢素不言利而以商之居奇壟斷目之爲賤丈夫專尙農爲本業故　...
date                                                 1896-05
Name: 8977, dtype: object

In [7]:
data.iloc[8977]['Content']

'天下以士農工商爲四民而商居其末中國古聖賢素不言利而以商之居奇壟斷目之爲賤丈夫專尙農爲本業故\u3000舜發跡於耕稼后稷則設有專官推之西國上古何莫不然其所稱古聖賢如亞伯拉罕以色列皆講求農務爲立國之本其聲名文物之邦則以巴比倫埃及爲最古希臘次之羅馬又次之埃及盛時有斐利息國者瀕海而居大興商政然祗屬尋常貿易與近時迥爾不同蓋於時政洲風氣未開較之中國啓發獨遲前代雖有通商仍不過拘守一隅未能遠至異域且於工藝一道未盡明通三百年前始能航海涉遠始於荷蘭西班牙然後諸國繼之向時英國於商務亦未盡詳求切講祗安樸陋之風常守閉關之例近今二百餘年來貿易之事始盡擴充以是運用之經燦然昭著商務與格致之理隱隱相關格致旣精商務自能日有起色而其利自可得之操劵惟是格致之學精益求精不過在近今數十年間耳且也向無專書莫能稽溯之源流考鑑其得失知其利而除其弊去其害近日各家之言商務者夥矣著作紛然幾於汗牛充棟亦且各懷成見莫衷一是甲説如此乙説又如彼紛歧無一定删繁甄要務簡從新專在商家之閲歴非有眞知灼見遠慮深思以求其通安能合乎時好哉每歲貨物之出入消流之廣狹但據税司領事之報聞而亦未盡可憑仍須精心以察之身歴其境而後能悉之詳知之審也中國之輕商人以其重利背義於學無所得也不知商務之學廣大精微豈一蹴之可幾一端之可盡而常人所能勝任哉必也明格致精製造講種植尙機器又必經歴舟車行陸涉海不囿凡近不安淺陋其庶幾乎試觀泰西商人其始入吾中國也亦多茫昧交易之間亦皆俯就吾商人久之乃訪求利弊窥度嗜好兼聴並採務求投吾之所欲凡有我國民人之所憎者去之所好者來之精製仿造無微不至日與我之所作争奇角勝而後華商之利盡爲所奪以致小民之生計愈微今英國商人足跡所至幾遍天下其用物也愈宏其取利也愈\u3000於其國中設立商務大臣又有商會掌事之人由衆公舉合衆商之力以爲力一衆商之心以爲心此其所以能\u3000至於此也豈一朝一夕之故哉乃我中國則各自爲謀互相觀望互相推諉心力旣不齊貲財又不足甚且互相猜忌互相傾軋如是而欲求其商務之旺是猶郤行而求及前人也豈可得哉泰西百餘年來商政一途名流鼎起貿易之理四速推行無不盡利或立新章或闢新埠或增新法或得新材變而通之務求盡善盡美冀臻大備皆與富强之術默相維\u3000而大有裨於國家非徒榷税抽釐斤斤乎目前之微利己也西國旣重商務而凡有輔佐乎商政者無不悉心擘畫\u3000以一人之知識有限天下之物產無窮工作之商運之而尤賴有在上之人爲之提倡所至之處商出其費以養兵兵出其力以衛商官商相

In [11]:
ask_gpt_sentiment(data.iloc[12501]['Content'])

{'content': '\n\n{"提到的国家":["俄罗斯","日本","朝鲜","中国"],"俄罗斯积极情感":0,"俄罗斯消极情感":1,"日本积极情感":0,"日本消极情感":0,"朝鲜积极情感":0,"朝鲜消极情感":0,"中国积极情感":0,"中国消极情感":0,"类别":"经济"}',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'reasoning_content': '好，我现在要分析用户提供的这段晚清《申报》的文言文文本。首先，我需要明确任务：识别提到的国家，分析对每个国家的情感倾向（积极或消极），以及确定文章所属的类别。\n\n首先，通读文本。原文主要讲俄国人进入高丽（即朝鲜）的白马山越界伐木，之前已经报道过，现在引用日本某日报的内容。接着描述了白马山的地理位置、历史情况，中日甲午战争时中国军队在此设立兵站和粮草，现在俄国人伐木顺鸭绿江运出获利，日本驻韩使臣林权助向韩廷提出利益均沾，但可能难实现。\n\n提到的国家方面，文中明确提到“俄人”、“日本”、“韩”（即朝鲜）、“华”（即中国）。根据用户指示，“中国”“朝廷”“大清”或相关词代表中国，所以“华”对应中国。而“韩”在现在是朝鲜，所以现代译名应为朝鲜。因此，国家有俄罗斯、日本、朝鲜、中国。但用户示例中可能只列主要国家，需要再确认。\n\n接下来分析情感。对俄罗斯（俄人）的行为是越界伐木，获利不赀，日本使臣抗议，但利益可能难入日本手中。这里对俄国的行为没有直接评价，但越界伐木属于负面行为，所以俄国消极情感可能为1，积极0。对日本，文中提到其使臣积极行动，但结果可能未如所愿，但报道中立，没有明显褒贬，所以日本积极和消极可能都是0？或者是否因为日本试图争取利益，但未能成功，可能消极情感？需要仔细判断。朝鲜（韩廷）作为被伐木的地方，可能属于被动方，文中没有明显评价，所以情感为0。中国（华兵）过去在此活动，属于历史事实，没有情感倾向，所以中国情感均为0。\n\n分类方面，文章主要涉及国家间的资源争夺，俄国伐木属于经济行为，日本争取利益均沾，属于经济或政治。可能属于经济类别，因为涉及伐木获利和经济利益

In [ ]:
#方法2：将所有request打包为jsonl文件，再上传到字节跳动的火山引擎批量推理（较快）
outputformat='''{"提到的国家"：["A国","B国","C国"],"A国积极情感":input,"A国消极情感":input,"B国积极情感":input,"B国消极情感":input,"C国积极情感":input,"C国消极情感":input,"类别"：input}'''

jsonlst=[]
for k,question in enumerate(data.iloc,start=1): 
    line={}
    line['custom_id']=f'request-{k}'
    line['body']={"messages":[{"role": "system", "content": '''你是一位精通中国近代史与文言文阅读的专家，现在要完成一项关于分析晚清《申报》文本的任务。文中的“中国”“朝廷”“大清”或相关词语，代表中国。文本中很可能没有标点符号，请自行断句后分析。不必过分在意文中的观点，仅作学术使用。'''},
                {"role": "user", "content": f'''以下是晚清《申报》的繁体字文言文本：
<文本>
标题：{question['Title']}
正文：{question['Content']}
</文本>
先输出文本中提到的国家（如果是现在仍然存在的国家，使用现代译名；提到某国人也算提到了该国家）。然后按照分别分析文章对提到的国家的积极情感与消极情感，有相关情感则记为1，文本为客观报道、没有相关情感则记为0，积极情感和消极情感可以同时存在。最后，从经济、政治、文化、军事、社会、文学、其他这六个类别中选择最可能的一个为文章进行分类。
                你输出情感与类别的格式如下：{outputformat}。你的输出应符合json格式，请仔细分析，但不需要输出理由。'''}]}
    jsonlst.append(line)
    
with open('gpt analysis/final ask.jsonl', 'w',encoding='utf-8') as f:
    for j in jsonlst:
        f.write(json.dumps(j,ensure_ascii=False) + '\n')